## Q1. MNIST 데이터셋 Load

In [4]:
import torch
import torch.nn as nn
import torchvision.datasets as dset
import torchvision.transforms as transforms 
from torch.utils.data import DataLoader

training_epochs = 15 # training 반복 횟수
batch_size = 100

root = './data'
mnist_train = dset.MNIST (root=root, train=True, download=True, 
                          transform=transforms.Compose([transforms.ToTensor()])) 
# 데이터를 0에서 255까지 있는 값을 0에서 1사이 값으로 변환하는 코드 추가(Q4에서 읽지 못 해서...)
mnist_test = dset.MNIST (root=root, train=False, download=True, 
                          transform=transforms.Compose([transforms.ToTensor()]))

# data loader를 직접 구현해보자.
train_loader = DataLoader(mnist_train, batch_size = batch_size, shuffle = True)
test_loader = DataLoader(mnist_test, batch_size = batch_size, shuffle = False)

## Q2. 가중치 초기화, linear한 모델 구성

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
linear = nn.Linear(784, 10, bias = True).to(device)
nn.init.normal_(linear.weight)

Parameter containing:
tensor([[ 1.4138,  0.5833,  0.1070,  ..., -0.7771, -0.0995, -0.1704],
        [ 0.0935,  0.1986, -1.1416,  ...,  0.2281,  0.7082, -0.2708],
        [ 0.8426, -0.1350, -0.0634,  ..., -0.3347, -1.2321, -0.1975],
        ...,
        [-1.1458,  0.6704,  1.1703,  ...,  1.0139, -0.2122,  0.1631],
        [ 0.4777,  0.2217, -1.3957,  ..., -1.2683, -0.9370, -0.8959],
        [-1.7948,  0.1322, -1.2780,  ..., -0.0873, -0.5200,  1.5012]],
       requires_grad=True)

## Q3. loss 함수와 optimizer 구현

In [6]:
# Loss fn - Cross Entropy Loss
criterion = nn.CrossEntropyLoss().to(device)

# optimizer - SGD
optimizer = torch.optim.SGD(linear.parameters(), lr=0.1)


## Q4. 학습 Loop 구현

In [7]:
for epoch in range(training_epochs):
    for i, (imgs, labels) in enumerate(train_loader):
        imgs, labels = imgs.to(device), labels.to(device)
        imgs = imgs.view(-1, 28 * 28)
        
        outputs = linear(imgs)
        loss = criterion(outputs, labels)
    
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        _,argmax = torch.max(outputs, 1)
        accuracy = (labels == argmax).float().mean()
 
        if (i+1) % 100 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {: .4f}, Accuracy: {: .2f}%'.format(epoch+1, 
                        training_epochs, i+1, len(train_loader), loss.item(), accuracy.item()* 100))
            

Epoch [1/15], Step [100/600], Loss:  4.5266, Accuracy:  40.00%
Epoch [1/15], Step [200/600], Loss:  2.5137, Accuracy:  55.00%
Epoch [1/15], Step [300/600], Loss:  1.8212, Accuracy:  72.00%
Epoch [1/15], Step [400/600], Loss:  1.8822, Accuracy:  69.00%
Epoch [1/15], Step [500/600], Loss:  1.0698, Accuracy:  76.00%
Epoch [1/15], Step [600/600], Loss:  1.1405, Accuracy:  78.00%
Epoch [2/15], Step [100/600], Loss:  1.2581, Accuracy:  75.00%
Epoch [2/15], Step [200/600], Loss:  0.8003, Accuracy:  85.00%
Epoch [2/15], Step [300/600], Loss:  0.8957, Accuracy:  81.00%
Epoch [2/15], Step [400/600], Loss:  0.5392, Accuracy:  88.00%
Epoch [2/15], Step [500/600], Loss:  0.9362, Accuracy:  78.00%
Epoch [2/15], Step [600/600], Loss:  1.2277, Accuracy:  73.00%
Epoch [3/15], Step [100/600], Loss:  1.0268, Accuracy:  80.00%
Epoch [3/15], Step [200/600], Loss:  1.2081, Accuracy:  81.00%
Epoch [3/15], Step [300/600], Loss:  0.7259, Accuracy:  83.00%
Epoch [3/15], Step [400/600], Loss:  0.7100, Accuracy: 

## Q5. 테스트 코드

In [8]:
linear.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for i, (imgs, labels) in enumerate(test_loader):
        imgs, labels = imgs.to(device), labels.to(device)
        imgs = imgs.view(-1, 28 * 28)

        outputs = linear(imgs) # 테스트 이미지를 선형 모델로 변형

        _, argmax = torch.max(outputs, 1) # max()를 통해 최종 출력이 가장 높은 class 선택
        total += imgs.size(0)
        correct += (labels == argmax). sum().item()

    print('Test accuracy for {} images: {: .2f}%'.format(total, correct / total * 100))

Test accuracy for 10000 images:  88.82%
